# HOLA

Intalamos pandas:

In [106]:
%pip install pandas
%pip install faker
%pip install pyreadline3


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [107]:
import pandas as pd 
import cmd, sys
import re
from IPython.display import clear_output
from faker import Factory # type: ignore
import pyreadline3

In [108]:


class Contact:
    def __init__(self, name, email, phone):
        self.name = name
        self.email = email
        self.phone = phone


    def __str__(self):
        return f"{self.name} | {self.email} | {self.phone}"
    

    def toDataFrame(self):
        return pd.DataFrame([self.__dict__])
       



In [109]:
class CapaPersistencia:
    # .csv
    def __init__(self, filename):
        self.filename = filename
        self.load()
    
    def load(self):
        try:
            self.data = pd.read_csv(self.filename)
        except:
            self.data = pd.DataFrame(columns=["name", "email", "phone"])
            
    def save(self):
        self.data.to_csv(self.filename, index=False)
        

    def insert(self, contact):
        self.data = pd.concat([self.data, contact.toDataFrame()], ignore_index=True)
        self.save()
    
    def delete(self, name):
        self.data = self.data[self.data["name"] != name]
        self.save()
    
    def modify(self, name, email, phone):
        self.data.loc[self.data["name"] == name, "email"] = email
        self.data.loc[self.data["name"] == name, "phone"] = phone
        self.save()

                

In [110]:
class ContactList:
    def __init__(self):
        self.contacts = []
        self.persistencia = CapaPersistencia("contacts.csv")
        self.load()

    def load(self):
        for index, row in self.persistencia.data.iterrows():
            self.contacts.append(Contact(row["name"], row["email"], row["phone"]))

    def save(self):
        for contact in self.contacts:
            self.persistencia.insert(contact)

    def add(self, contact):
        self.contacts.append(contact)
        self.persistencia.insert(contact)
        
    def modify(self, name, contact):
        for c in self.contacts:
            if c.name == name:
                c.email = contact.email
                c.phone = contact.phone
                break
        self.persistencia.modify(contact.name, contact.email, contact.phone)
        
    def delete(self, name):
        for c in self.contacts:
            if c.name == name:
                self.contacts.remove(c)
                self.persistencia.delete(name)
                break

    def clear(self):
        self.contacts = []
        self.persistencia.data = pd.DataFrame(columns=["name", "email", "phone"])
        self.persistencia.save()
    

In [111]:
class SearchEngine:

    def __init__(self, contacts, searchTerm = "", page = 0, regex = False):
        self.contacts = contacts
        self.searchTerm = searchTerm
        self.page = page
        self.items_per_page = 10
        self.total_items = -1
        self.regex = regex

    def search(self):
        found = []
        if self.searchTerm == "":

            self.total_items = len(self.contacts)

            return self.contacts[self.page * self.items_per_page: (self.page + 1) * self.items_per_page]
        if self.regex:
            for contact in self.contacts:
                if re.match(self.searchTerm, contact.name) or re.match(self.searchTerm, contact.email) or re.match(self.searchTerm, contact.phone):
                    found.append(contact)
        else:
            term = self.searchTerm.lower()
            for contact in self.contacts:
                phone = contact.phone
                phone = phone.lower().replace(" ", "")
                if term in contact.name.lower() or term in contact.email.lower() or term.replace(" ", "") == phone:
                    found.append(contact)
        self.total_items = len(found)
        return found[self.page * self.items_per_page: (self.page + 1) * self.items_per_page]
    
    def print(self):

        contacts = self.search()
        print("Number of contacts: " + str(self.total_items))
        for i in range(len(contacts)):
            print(str(i) + ". " + str(contacts[i]))
        
        print("Página " + str(self.page + 1) + " de " + str(self.total_items // self.items_per_page))
        
        
        

In [112]:
class ContactShell(cmd.Cmd):
    intro = "Agenda de Contactos\nescribe ? para ver una lista de los comandos\n"
    prompt = "contact> "
    
    def cmdloop(self, intro=None):
            """Repeatedly issue a prompt, accept input, parse an initial prefix
            off the received input, and dispatch to action methods, passing them
            the remainder of the line as argument.

            """

            self.preloop()
            if self.use_rawinput and self.completekey:
                try:
                    import readline
                    self.old_completer = readline.get_completer()
                    readline.set_completer(self.complete)
                    readline.parse_and_bind(self.completekey+": complete")
                except ImportError:
                    pass
            try:
                if intro is not None:
                    self.intro = intro
                if self.intro:
                    self.stdout.write(str(self.intro)+"\n")
                stop = None
                while not stop:
                    if self.cmdqueue:
                        line = self.cmdqueue.pop(0)
                    else:
                        if self.use_rawinput:
                            try:
                                line = input(self.prompt)
                                print("input: "+ line)
                            except EOFError:
                                line = 'EOF'
                        else:
                            self.stdout.write(self.prompt)
                            self.stdout.flush()
                            line = self.stdin.readline()
                            if not len(line):
                                line = 'EOF'
                            else:
                                line = line.rstrip('\r\n')
                    line = self.precmd(line)
                    stop = self.onecmd(line)
                    stop = self.postcmd(stop, line)
                self.postloop()
            finally:
                if self.use_rawinput and self.completekey:
                    try:
                        import readline
                        readline.set_completer(self.old_completer)
                    except ImportError:
                        pass



    def __init__(self):
        super().__init__()
        self.contacts = ContactList()
        self.search_engine = None
        self.pressed_enter = False
        
    def postcmd(self, stop: bool, line: str) -> bool:
        print("stop")
        return super().postcmd(stop, line)


    def precmd(self, line: str) -> str:
        print(line)
        if not line:
            return line
        
        if not (line in ["modify", "delete"] and self.search_engine):
            self.clear()
        print(line)
        if line:
            self.pressed_enter = False
        
        if line.split()[0] not in ["search", "list", "regex","modify", "delete"]:
            print(line)
            self.search_engine = None
        
        return super().precmd(line)

    def emptyline(self) -> bool:
        if(self.search_engine):
            if self.search_engine.page * self.search_engine.items_per_page + self.search_engine.items_per_page < self.search_engine.total_items:
                self.clear()
                self.search_engine.page += 1
                self.search_engine.print()
                print()
                print("'search <term> <page>' para ver mas contactos o pulsa enter para ir a la pagina siguiente")
                print("'modify <index>' para modificar un contacto")
                print("'delete <index>' para borrar un contacto")
    
            else:
                print("No hay mas contactos")
                self.search_engine = None
            return False



        if self.pressed_enter:
            return True
        self.pressed_enter = True
        print("Escribe un comando valido o pulsa enter otra vez para salir.")
        self.do_help("")
        return False
       
    def do_list(self, page = 0):
        'Lista de todos los contactos: list <page?>'
        #self.clear()
        if not page:
            page = 0
        else:
            try:
                page = int(page) - 1
            except:
                print("No has introducido un numero valido")
                return
        self.search_engine = SearchEngine(self.contacts.contacts, page = page)
        self.search_engine.print()
        print()
        print("'list <page>' para ver mas contactos o pulsa enter para ir a la pagina siguiente")
        print("'modify <index>' para modificar un contacto")
        print("'delete <index>' para borrar un contacto")
        
        
                
    def do_fakedata(self, args):
        'Genera contactos falsos: fakedata <num?>'
        num = 100
        
        if args:
            try:
                num = int(args)
            except:
                print("No has introducido un numero valido")
                return
            
        self.contacts.clear()

        fake = Factory.create("es_ES")
        for _ in range(num):
            
            phone = fake.phone_number()
            phone = phone.replace(" ", "").replace("-", "").replace("(", "").replace(")", "")
            
            phone = phone[:3] + " " + phone[3:6] + " " + phone[6:9] + " " + phone[9:]
            
            
            
            
            
            contact = Contact(fake.name(), fake.email(), phone)
            self.contacts.add(contact)

        
                
    def do_search(self, args):
        'Search for a contact'
        
        #args[0] is search term and args[1] is page number(optional)
        
        args = args.split()
        if not args:
            print("usage: search <term> <page>")
            return
        term = args[0]
        page = 0
        if len(args) > 1:
            try:
                page = int(args[1]) -1
            except:
                print("Invalid page number")
                return
        self.search_engine = SearchEngine(self.contacts.contacts, term, page)
        self.search_engine.print()
        print()
        print("'search <term> <page>' para ver mas contactos o pulsa enter para ir a la pagina siguiente")
        print("'modify <index>' para modificar un contacto")
        print("'delete <index>' para borrar un contacto")
            
            
    def do_regex(self, args):
        'Buscar un contact usando regex: regex <regex> <page>'
        
        args = args.split()
        if not args:
            print("usage: regex <term> <page>")
            return
        term = args[0]
        page = 0
        if len(args) > 1:
            try:
                page = int(args[len(args) - 1]) -1
                term = " ".join(args[:-1])
            except:
                term = " ".join(args)
        self.search_engine = SearchEngine(self.contacts.contacts, term, page, regex = True)
        self.search_engine.print()
        print()
        print("'regex <regex> <page>' para ver mas contactos o pulsa enter para ir a la pagina siguiente")
        print("'modify <index>' para modificar un contacto")
        print("'delete <index>' para borrar un contacto")
            
    def do_modify(self, index):
        'Modifica un contacto: modify <index>'
        if not index:
            print("usage: modify <index>")
            return
        if not self.search_engine:
            print("Please search for a contact first use 'search' or 'list'")
            return
        try:
            index = int(index)
        except:
            print("Invalid index")
            return
        contacts = self.search_engine.search()
        if index < 0 or index >= len(contacts):
            print("Invalid index")
            return
        contact = contacts[index]
        print("Contact found: " + str(contact))
        print("Enter new data")
        email = input("Email (blank to keep current) : ")
        phone = input("Phone (blank to keep current): ")
        
        if not email:
            email = contact.email
        if not phone:
            phone = contact.phone
        
        
        contact = Contact(contact.name, email, phone)
        self.contacts.modify(contact.name, contact)
        
    def do_delete(self, index):
        'Borra un contacto: delete <index>'
        if not index:
            print("usage: delete <index>")
            return
        if not self.search_engine:
            print("Please search for a contact first use 'search' or 'list'")
            return
        try:
            index = int(index)
        except:
            print("Invalid index")
            return
        contacts = self.search_engine.search()
        if index < 0 or index >= len(contacts):
            print("Invalid index")
            return
        contact = contacts[index]
        print("Contact found: " + str(contact))
        confirm = input("Are you sure you want to delete this contact? (yes/no) ")
        if confirm == "yes":
            self.contacts.delete(contact.name)
        else:
            print("Contact not deleted")
        
        
        

    def do_add(self, args):
        'Crea un contacto: add <name> <email> <phone>'
        if not args:
            print("usage: add <name> <email>")
            return
        name, email = args.split()
        contact = Contact(name, email)
        self.contacts.add(contact)

    def do_exit(self, args):
        'Exit the program'
        return True
    
    
    def default(self, line: str) -> None:
        print(f"Sintaxis Desconocida: {line}")
        self.do_help("")
    
    def clear(self): 
        print("Cleared")
    

In [113]:

if __name__ == '__main__': 
    shell = ContactShell()
    shell.cmdloop()
    #cli = CLI()
    #cli.run()


Agenda de Contactos
escribe ? para ver una lista de los comandos

input: list
list
Cleared
list
Number of contacts: 100
0. Eugenio Múgica Seguí | cocanando@example.com | +34 874 781 942
1. Nilda de Lago | nereidacabeza@example.net | +34 985 731 851
2. Atilio de Pagès | ale05@example.com | +34 806 492 890
3. Ariel Perez Escobar | jvidal@example.org | +34 827 443 864
4. María Luisa Elba Soriano Miralles | rosalia35@example.net | +34 873 195 882
5. Ruben Álamo Feijoo | manola73@example.net | +34 944 424 147
6. Héctor José Sandoval Torrents | artigaseulalia@example.net | +34 967 258 878
7. Patricio Andrade Jover | pozosabas@example.com | +34 803 407 249
8. Teófila Escamilla Carvajal | humbertobellido@example.org | +34 812 547 648
9. Humberto Serna Cerezo | heriberto01@example.net | +34 887 316 428
Página 1 de 10

'list <page>' para ver mas contactos o pulsa enter para ir a la pagina siguiente
'modify <index>' para modificar un contacto
'delete <index>' para borrar un contacto
stop
input: 
